# Homework 10: Cleaning data with Regular Expressions

Time to use regular expressions!

# Hints and notes

### Opening files in subdirectories

Notice that this notebook might be **homework/**, but!!! the csvs and text files might be in **homework/scraped/** or **/homework/scraped/minutes_pdfs** or **/homework/pdfs/**. To open a file in a subdirectory, instead of having the filename be `"file.csv"` you'll just use `"some/subfolder/file.csv"`

### Opening text files

This will open up a file, read it in and show you the first 500 characters.

```python
contents = open("your-filename.txt").read()
contents[0:500]
```

> You might need `open("your-filename.txt", encoding="utf8").read()`

### Using regex

For some dumb reason you need to put `r` in front of the string you use when you're talking about regex. Just plain `"(\d\d\d)"` will usually work, but *sometimes* it won't and you'll need `r"(\d\d\d)`. It's best to just use the `r` all of the time, if you can remember!

### Using `.str.extract`

When you use `.str.extract`, you're always going to **capture one thing** and save it to a new column. You need to wrap the things you're interested in with parenthesis `(` `)`.

```python
df['phone_number'] = df['old_column'].str.extract(r"My phone number is (\d\d\d-\d\d\d-\d\d\d\d)")
```

### Setting pandas options

Pandas has a lot of options, like how many columns or rows it will show you, or how many characters it will show in a column before it stops showing you anything. Here are a few useful ones:

* `display.max_cols`: Number of columns to show at once
* `display.max_rows`: Number of rows to show at once
* `display.max_colwidth`: Maximum number of characters displayed from a string

You can set them using `pd.set_option("display.max_rows", 1000)`, for example, to show 1000 rows at a time. You can find a lot more at https://pandas.pydata.org/pandas-docs/stable/generated/pandas.set_option.html

### Regular expressions reference

I personally think http://www.regular-expressions.info/ is a wonderful wonderful reference (and tutorial), even if it's ugly! But here's a quick reference for you:

* `\d` is a digit
* `\d*` is zero or more digits 
* `\d+` is one or more digits
* `.` matches anything for ONE character
* `.*` is "give me anything forever"
* `\s` is whitespace, a.k.a. spaces and tabs
* `\w` is a word character, which includes capital and lowercase letters, numbers and hyphens.
* You can put `*` after anything, so `\w*` would mean "as many word characters as you can find"
* `\b` is a word boundary (you'll need the `r""` thing for this one)
* `( )` is a "capture group" for saving something
* `\1` is used when doing find/replace to say "put the first captured group here" (note, it's a dollar sign instead of a backslash in some editors)
* `[ABCDE]` is a character class, which means "match one of these, I don't care which"
* dollar sign means "end of the line"
* caret ^ means "beginning of the line"
* `\.` means "no really seriously I mean a period not just anything"
* You can use `\` with anything else that would normally be a special character, too, not just periods. `(` or `[` or whatever.

### Cleaning up extracted columns

Sometimes you get `\n` (newlines) or spaces or `\t` (tabs) or stuff at the beginning or the end of your column. `.str.strip()` will usually take care of that, just attach it after your `.str.extract()`

After you extract something, it's still a string even though you look at it and know it's a number. Use `.astype(int)` to turn it into an integer (no decimal) or `.astype(float)` to turn it into a float (yes decimal)

### Writing regular expressions in general

Even if I'm using regex in pandas or Python, I like to test them in my text editor with "Find." The highlighting really helps me see if I'm matching things! I also like to think "what stays the same?" when designing patterns, write those parts first, then fill in the blanks with what I want to capture.

## Importing

There might be more, I just wanted to put this up here for the `pd.set_option` part. It allows you to see a lot of content in a single column of pandas, which will be important for some parts below.

In [1]:
import re
import pandas as pd
pd.set_option('display.max_colwidth', 500)
import numpy as np

# Part 1: Using `.str.extract` to pull data from columns in pandas

## 1.1 H&M

Open up `hm.csv` from the `scraped` directory. I want **four new columns**:

1. `price_original`, the original price, one of the new price
2. `price_discounted`, the discounted price
3. `pct_discount`, the percent discount
4. `article_id`, the article id (from the url)

Save as **hm_cleaned.csv**.

**Note:** When you look at it, it... won't look right. I don't know why, pandas is weird. Look at the `price` column by itself using `df['price']` before you write your regex.

**Tip:** Remember that `$` is a special regex symbol! You might need to escape it.

**Tip:** When doing `.str.extract`, the whole match doesn't get captured, only what you put `()` around! Think about anchoring to different points of the string, or things in the string.

**Tip:** Not all prices have cents!

**Tip:** Your first instinct about how to compute the percent discount is probably wrong

In [2]:
df = pd.read_csv('scraped/hm.csv', encoding='utf-8')

In [3]:
df.head()

,name,price,url
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A


In [4]:
df['new_price'] = df.price.str.extract(r"(\d?\d?\d\.\d\d)").astype(float)
df['old_price'] = df.price.str.extract(r" \$(.*)$").astype(float)
df = df.drop('price', 1)

In [5]:
df['dicsount'] = (df['old_price']-df['new_price'])/df['old_price']*100

In [6]:
df['article_id'] = df.url.str.extract(r".*le=(\d+)-")
df.head()

,name,url,new_price,old_price,dicsount,article_id
0,Washed Linen Duvet Cover Set,http://www.hm.com/us/product/13472?article=13472-N,59.99,129.00,53.496124,13472
1,Candle in Glass Jar,http://www.hm.com/us/product/35079?article=35079-D,6.99,17.99,61.145081,35079
2,Glittery Cushion Cover,http://www.hm.com/us/product/72462?article=72462-A,7.99,17.99,55.586437,72462
3,Textured-weave Cushion Cover,http://www.hm.com/us/product/58926?article=58926-C,6.99,12.99,46.189376,58926
4,Stoneware Bowl,http://www.hm.com/us/product/74242?article=74242-A,17.99,24.99,28.011204,74242


## 1.2 Sci-Fi Authors

Open up `sci-fi.csv` to clean. Get rid of the `\n` on the title and and give me six new columns:

* `avg_rating`
* `rating_count`
* `total_score`
* `score_votes`
* `series` the series the book belongs to
* `series_no` the book in the series that it is

For series, I'm talking about e.g. `(The Hunger Games, #1)` is `series` "The Hunter Games" and `series_no` 1.

Save as **sci-fi_cleaned.csv**.

**Tip:** You don't need regex to clean the title - there's a special thing that removes whitespace from the beginning/end of strings

**Tip:** Remember that `(` and `)` are special characters

**BONUS:** When you make the `total_score` column, pay close attention to it. If you notice the problem, fix it.

**BONUS:** You don't need these columns to be numbers, but life would be better if they were. 

In [7]:
df = pd.read_csv('scraped/sci-fi.csv')
df['title'] = df.title.str.strip()
df.head()

,full_rating,full_score,rank,title,url
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,The Handmaid's Tale,/book/show/38447.The_Handmaid_s_Tale
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"Frankenstein, or The Modern Prometheus",/book/show/18490.Frankenstein_or_The_Modern_Prometheus
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"A Wrinkle in Time (A Wrinkle in Time Quintet, #1)",/book/show/18131.A_Wrinkle_in_Time
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,The Left Hand of Darkness,/book/show/18423.The_Left_Hand_of_Darkness


In [8]:
df['rating_avg'] = df.full_rating.str.extract(r"^ (\d?\d\.\d\d)").astype(float)
df['rating_count'] = df.full_rating.str.extract(r"— (.*?\d+) ratings$")
df['rating_count'] = df.rating_count.str.replace(',','')
df['total_score'] = df.full_score.str.extract(r"score: (\d*,?\d+)")
df['score_votes'] = df.full_score.str.extract(r"(\d+,?\d+) people")
df['series'] = df.title.str.extract(r"\((.*),.*\)")
df['series_no'] = df.title.str.extract(r"#(\d+)")

In [9]:
df = df.drop('full_rating', 1)
df = df.drop('full_score', 1)
df.head()

,rank,title,url,rating_avg,rating_count,total_score,score_votes,series,series_no
0,1,The Handmaid's Tale,/book/show/38447.The_Handmaid_s_Tale,4.07,785502,"28,539",292,NaN,NaN
1,2,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games,4.34,5212935,"27,566",282,The Hunger Games,1
2,3,"Frankenstein, or The Modern Prometheus",/book/show/18490.Frankenstein_or_The_Modern_Prometheus,3.76,922308,"20,049",205,NaN,NaN
3,4,"A Wrinkle in Time (A Wrinkle in Time Quintet, #1)",/book/show/18131.A_Wrinkle_in_Time,4.04,702272,"17,684",185,A Wrinkle in Time Quintet,1
4,5,The Left Hand of Darkness,/book/show/18423.The_Left_Hand_of_Darkness,4.06,77664,"16,070",165,NaN,NaN


## 1.3 Where you're just doing one of my former students' projects

Once upon a time my student Stefan did a project that involved some lawyer stuff. Most of the content was in PDFs, though! I converted them to text files and put them into the `pdfs` folder, and gave you code below to open up each of them and save their contents into a dataframe.

What a nice dataframe! I want you to add the following columns to it:

* `lawyer_app`, the applicant's lawyer (pro se means that they did it themselves, that's fine)
* `lawyer_gov`, the government's lawyer
* `judge`, the name of the judge
* `access`, whether the clearance is granted or denied (although you might miss a few)

Save as **court_cleaned.csv**.

**Note:** You can look at the original PDFs, they're also included.

**Note:** This uses a fun utility called `glob`, which is mostly fun because you use it as `glob.glob`. It's used to find files that match a certain filename pattern.

**BONUS:** You'll be happy once you get the judge, but make sure it doesn't have any extra punctuation on it.

**BONUS:** You can for some words using `.str.contains("blah")` and save it into new columns. Maybe `has_debt`, `has_bankruptcy`, etc.

> It's okay if it isn't perfect. Converting PDF into data rarely is! Usually you get 90% of it done with computers, then send people to enter the other 10% by hand.

In [10]:
import glob
filenames = glob.glob("pdfs/*.txt")
contents = [open(filename, encoding="utf8").read() for filename in filenames]
df = pd.DataFrame({'filename': filenames, 'content': contents})
df.shape

(15, 2)

In [11]:
df.head()

,filename,content
0,pdfs/11-04909.h1.pdf.txt,"\n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n DEPARTMENT OF DEFENSE \n\n \n \n\n \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n \n\n \n\nISCR Case No. 11-04909 \n\n \n\nFor Government: Richard Stevens, Esq., Department Counsel \n\nFor Applicant: Pro se \n\n \n\n \nDUFFY, James F., Administrative Judge: \n\n \nApplicant mitigated \n\nconsiderations). Clearance is ..."
1,pdfs/11-08313.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n\nISCR Case No. 11-08313 \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n--------------- \n \n\n \n\n \n \n\n) \n) \n) \n) \n) \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\nFor Government: Julie R. Mendez, Esquire, Department Counsel \n\nFo..."
2,pdfs/11-11916.h1.pdf.txt,"\n\nDEPARTMENT OF DEFENSE \n\nDEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\nREDACTED \n\n \n\nISCR Case No. 11-11916 \n\nMENDEZ, Francisco, Administrative Judge: \n\n \nApplicant did not mitigate security concerns raised by his exercise of foreign \ncitizenship, including the possession of a current foreign passpor..."
3,pdfs/11-03073.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n\n \n\nISCR Case No. 11-03073 \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n\n \n\n \n \n\nFor Government: Robert J. Kilmartin, Esq., Department Counse..."
4,pdfs/12-00038.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \n \nApplicant for Security Clearance \n\n \n \n\n) \n) \n) \n) \n) \n) \n \n \n\n \n \n\nISCR Case No. 12-00038 \n \n\nAppearances \n\nFor Government: Robert Kilmartin, Esq., Department Counsel \n\nFor Applicant: Pro Se \n\n \n\nApril 28, 2017 \n\n__________..."


Okay, now do the work and **make those new columns!**

In [12]:
def get_judge(row):
    regex_1 = r"([A-Z]+, [A-Z].*), \w+ Judge:"
    regex_2 = r"Judge +([A-Z]\w+ ?[A-Z]?.? ?[A-Z]\w+)"
    if re.findall(regex_1, row['content']):
        return re.findall(regex_1, row['content'])[0]
    elif re.findall(regex_2, row['content']):
        return re.findall(regex_2, row['content'])[0]
    else:
        return 'not found'

In [13]:
df['judge'] = df.apply(get_judge, axis = 1)
df.head()

,filename,content,judge
0,pdfs/11-04909.h1.pdf.txt,"\n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n DEPARTMENT OF DEFENSE \n\n \n \n\n \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n \n\n \n\nISCR Case No. 11-04909 \n\n \n\nFor Government: Richard Stevens, Esq., Department Counsel \n\nFor Applicant: Pro se \n\n \n\n \nDUFFY, James F., Administrative Judge: \n\n \nApplicant mitigated \n\nconsiderations). Clearance is ...","DUFFY, James F."
1,pdfs/11-08313.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n\nISCR Case No. 11-08313 \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n--------------- \n \n\n \n\n \n \n\n) \n) \n) \n) \n) \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\nFor Government: Julie R. Mendez, Esquire, Department Counsel \n\nFo...","MARSHALL, Jr., Arthur E."
2,pdfs/11-11916.h1.pdf.txt,"\n\nDEPARTMENT OF DEFENSE \n\nDEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\nREDACTED \n\n \n\nISCR Case No. 11-11916 \n\nMENDEZ, Francisco, Administrative Judge: \n\n \nApplicant did not mitigate security concerns raised by his exercise of foreign \ncitizenship, including the possession of a current foreign passpor...","MENDEZ, Francisco"
3,pdfs/11-03073.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n\n \n\nISCR Case No. 11-03073 \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n\n \n\n \n \n\nFor Government: Robert J. Kilmartin, Esq., Department Counse...","LOUGHRAN, Edward W."
4,pdfs/12-00038.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \n \nApplicant for Security Clearance \n\n \n \n\n) \n) \n) \n) \n) \n) \n \n \n\n \n \n\nISCR Case No. 12-00038 \n \n\nAppearances \n\nFor Government: Robert Kilmartin, Esq., Department Counsel \n\nFor Applicant: Pro Se \n\n \n\nApril 28, 2017 \n\n__________...","GOLDSTEIN, Jennifer I."


In [14]:
def get_lawyers(row):
    lawyer = {}
    regex_app = r"For Applicant: +([A-Z]\w+ [A-Z]?\.? ?\w+)"
    regex_gov = r"For Government: +([A-Z]\w+ [A-Z]?\.? ?\w+)"
    regex_gov_2 = "FOR GOVERNMENT\n+([A-Z]\w+ [A-Z]?\.? ?\w+)"
    regex_app_2 = "FOR APPLICANT\n+([A-Z]\w+ [A-Z]?\.? ?\w+)"
    if re.findall(regex_app, row['content']):
        lawyer['lawyer_app'] = re.findall(regex_app, row['content'])[0]
        lawyer['lawyer_gov'] = re.findall(regex_gov, row['content'])[0]
    elif re.findall(regex_app_2, row['content']):
        lawyer['lawyer_app'] = re.findall(regex_app_2, row['content'])[0]
        lawyer['lawyer_gov'] = re.findall(regex_gov_2, row['content'])[0]
    else:
        lawyer['lawyer_app'] = 'not found'
        lawyer['lawyer_gov'] = 'not found'
    return pd.Series(lawyer)

In [15]:
df.apply(get_lawyers, axis = 1)

,lawyer_app,lawyer_gov
0,Pro se,Richard Stevens
1,Pro se,Julie R. Mendez
2,Pro se,David F. Hayes
3,Mark S. Zaid,Robert J. Kilmartin
4,Pro Se,Robert Kilmartin
5,Stephen Glassman,Erin Thompson
6,Mark S. Zaid,Julie R. Mendez
7,Pro se,James B. Norman
8,Ryan C. Nerney,Richard Stevens
9,Pro se,James B. Norman


In [16]:
df = df.apply(get_lawyers, axis = 1).join(df)
df

,lawyer_app,lawyer_gov,filename,content,judge
0,Pro se,Richard Stevens,pdfs/11-04909.h1.pdf.txt,"\n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n DEPARTMENT OF DEFENSE \n\n \n \n\n \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n \n\n \n\nISCR Case No. 11-04909 \n\n \n\nFor Government: Richard Stevens, Esq., Department Counsel \n\nFor Applicant: Pro se \n\n \n\n \nDUFFY, James F., Administrative Judge: \n\n \nApplicant mitigated \n\nconsiderations). Clearance is ...","DUFFY, James F."
1,Pro se,Julie R. Mendez,pdfs/11-08313.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n\nISCR Case No. 11-08313 \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n--------------- \n \n\n \n\n \n \n\n) \n) \n) \n) \n) \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\nFor Government: Julie R. Mendez, Esquire, Department Counsel \n\nFo...","MARSHALL, Jr., Arthur E."
2,Pro se,David F. Hayes,pdfs/11-11916.h1.pdf.txt,"\n\nDEPARTMENT OF DEFENSE \n\nDEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\nREDACTED \n\n \n\nISCR Case No. 11-11916 \n\nMENDEZ, Francisco, Administrative Judge: \n\n \nApplicant did not mitigate security concerns raised by his exercise of foreign \ncitizenship, including the possession of a current foreign passpor...","MENDEZ, Francisco"
3,Mark S. Zaid,Robert J. Kilmartin,pdfs/11-03073.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n\n \n\nISCR Case No. 11-03073 \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n\n \n\n \n \n\nFor Government: Robert J. Kilmartin, Esq., Department Counse...","LOUGHRAN, Edward W."
4,Pro Se,Robert Kilmartin,pdfs/12-00038.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \n \nApplicant for Security Clearance \n\n \n \n\n) \n) \n) \n) \n) \n) \n \n \n\n \n \n\nISCR Case No. 12-00038 \n \n\nAppearances \n\nFor Government: Robert Kilmartin, Esq., Department Counsel \n\nFor Applicant: Pro Se \n\n \n\nApril 28, 2017 \n\n__________...","GOLDSTEIN, Jennifer I."
5,Stephen Glassman,Erin Thompson,pdfs/12-01787.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n\n \n\nISCR Case No. 12-01787 \n\nAppearances \n\nFor Government: Erin Thompson, Esq., Department Counsel \n\nFor Applicant: Stephen Glassman, Esq. \n\n______________ \n\n \nDecision \n\n______________ \n\nGARCIA, Candace Le’...","GARCIA, Candace Le’i"
6,Mark S. Zaid,Julie R. Mendez,pdfs/11-07728.h1.pdf.txt,"DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n------------------------ \n \n\n \n\nISCR Case No. 11-07728 \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n\n \n \n\nAppearances \n\n___________ \n\n \nDecision \n\n___________ \n\nFor Government: Julie R. Mendez, Esq., Department Counsel \n\nFor Applicant: Mark S. Zaid, Esq. \n\n \n\nHARVEY, Mark, Administrative Judge: \n \...","HARVEY, Mark"
7,Pro se,James B. Norman,pdfs/12-01601.a1.pdf.txt,"KEYWORD: Guideline F\n\nDIGEST: Most of the document Applicant submitted on appeal were not previously submitted to\nthe Judge and the Board is prohibited from receiving or considering them. Adverse decision\naffirmed.\n\nCASENO: 12-01601.a1\n\nDATE: 09/23/2016\n\nIn Re:\n\n-------\n\nApplicant for Security Clearance\n\nDATE: September 23, 2016\n\nISCR Case No. 12-01601\n\n)\n)\n)\n)\n)\n)\n)\n)\n\nAPPEAL BOARD DECISION\n\nAPPEARANCES\n\nFOR GOVERNMENT\n\nJames B. Norman, Esq., Chief Depart...",Martin H. Mogul
8,Ryan C. Nerney,Richard Stevens,pdfs/11-12635.h1.pd

In [17]:
def get_decision(row):
    decision = {}
    reasons = ['alcohol', 'debts', 'DUI', 'OUI', 'bankruptcy', 'Financial Considerations', 'Foreign Influence']
    all_reasons = []
    if re.findall('is granted', row['content']):
        decision['decision'] = 'granted'
        decision['reason'] = np.nan
    elif re.findall('is denied', row['content']) or re.findall('declined to grant', row['content']):
        decision['decision'] = 'denied'
        for reason in reasons:
            if re.findall(reason, row['content'], re.I):
                all_reasons.append(reason)                
        decision['reason'] = ', '.join(all_reasons)
    else:
        decision['decision'] = 'not found'
        decision['reason'] = 'not found'
    return pd.Series(decision)


In [18]:
df = df.apply(get_decision, axis = 1).join(df)
df

,decision,reason,lawyer_app,lawyer_gov,filename,content,judge
0,granted,NaN,Pro se,Richard Stevens,pdfs/11-04909.h1.pdf.txt,"\n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n DEPARTMENT OF DEFENSE \n\n \n \n\n \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n \n\n \n\nISCR Case No. 11-04909 \n\n \n\nFor Government: Richard Stevens, Esq., Department Counsel \n\nFor Applicant: Pro se \n\n \n\n \nDUFFY, James F., Administrative Judge: \n\n \nApplicant mitigated \n\nconsiderations). Clearance is ...","DUFFY, James F."
1,denied,Foreign Influence,Pro se,Julie R. Mendez,pdfs/11-08313.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n\nISCR Case No. 11-08313 \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n--------------- \n \n\n \n\n \n \n\n) \n) \n) \n) \n) \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\nFor Government: Julie R. Mendez, Esquire, Department Counsel \n\nFo...","MARSHALL, Jr., Arthur E."
2,denied,Foreign Influence,Pro se,David F. Hayes,pdfs/11-11916.h1.pdf.txt,"\n\nDEPARTMENT OF DEFENSE \n\nDEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\nREDACTED \n\n \n\nISCR Case No. 11-11916 \n\nMENDEZ, Francisco, Administrative Judge: \n\n \nApplicant did not mitigate security concerns raised by his exercise of foreign \ncitizenship, including the possession of a current foreign passpor...","MENDEZ, Francisco"
3,granted,NaN,Mark S. Zaid,Robert J. Kilmartin,pdfs/11-03073.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n\n \n\nISCR Case No. 11-03073 \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n\n \n\n \n \n\nFor Government: Robert J. Kilmartin, Esq., Department Counse...","LOUGHRAN, Edward W."
4,denied,"debts, Financial Considerations, Foreign Influence",Pro Se,Robert Kilmartin,pdfs/12-00038.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \n \nApplicant for Security Clearance \n\n \n \n\n) \n) \n) \n) \n) \n) \n \n \n\n \n \n\nISCR Case No. 12-00038 \n \n\nAppearances \n\nFor Government: Robert Kilmartin, Esq., Department Counsel \n\nFor Applicant: Pro Se \n\n \n\nApril 28, 2017 \n\n__________...","GOLDSTEIN, Jennifer I."
5,granted,NaN,Stephen Glassman,Erin Thompson,pdfs/12-01787.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n\n \n\nISCR Case No. 12-01787 \n\nAppearances \n\nFor Government: Erin Thompson, Esq., Department Counsel \n\nFor Applicant: Stephen Glassman, Esq. \n\n______________ \n\n \nDecision \n\n______________ \n\nGARCIA, Candace Le’...","GARCIA, Candace Le’i"
6,granted,NaN,Mark S. Zaid,Julie R. Mendez,pdfs/11-07728.h1.pdf.txt,"DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n------------------------ \n \n\n \n\nISCR Case No. 11-07728 \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n\n \n \n\nAppearances \n\n___________ \n\n \nDecision \n\n___________ \n\nFor Government: Julie R. Mendez, Esq., Department Counsel \n\nFor Applicant: Mark S. Zaid, Esq. \n\n \n\nHARVEY, Mark, Administrative Judge: \n \...","HARVEY, Mark"
7,denied,"debts, bankruptcy, Financial Considerations",Pro se,James B. Norman,pdfs/12-01601.a1.pdf.txt,"KEYWORD: Guideline F\n\nDIGEST: Most of the document Applicant submitted on appeal were not previously submitted to\nthe Judge and the Board is prohibited from receiving or considering them. Adverse decision\naffirmed.\n\nCASENO: 12-01601.a1\n\nDATE: 09/23/2016\n\nIn Re:\n\n-------\n\nApplicant for Security Clearance\n\nDATE: September 23,

# Reading books

When you're doing text work, you're legally obligated work on Jane Austen's Pride and Prejudice (at least I *think* so). Let's do some naive analysis of it!

## Read in Jane Austen's Pride and Prejudice (without moving the file!)

It's in the `data/` directory, and named `Austen_Pride.txt`.

In [23]:
contents = open('data/Austen_Pride.txt').read()

## Look at the first 500 or so characters of it 

In [28]:
contents[0:500]

' Pride and Prejudice\nby Jane Austen\nChapter 1\nIt is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.\nHowever little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.\n"My dear Mr. Bennet," said his lady to him one day, "have you heard that Nethe'

## Use a regular expression to find every "he" or "she" in the book. There should be about 3000 of them.

**Tip:** Do you know about **word boundaries?** `\b` means "the beginning of end of a word."

**Tip:** You might also want to use `re.IGNORECASE`. Maybe you'll need to google it? 

**Tip:** Do NOT use `re.compile`

In [42]:
he_count = len(re.findall(r"(\bhe\b)", contents, re.I))
she_count = len(re.findall(r"(\bshe\b)", contents, re.I))
pronoun_count = len(re.findall(r"(\bs?he\b)", contents, re.I))
print(he_count, she_count, pronoun_count)

1338 1709 3047


## Use a regular expression to find those same "he" or "she"s, but also match *the word after it*

The first four should be:

* he is
* he had
* she told
* he came

In [43]:
he_next = re.findall(r"(\bhe\b\W+\w+)", contents, re.I)
she_next = re.findall(r"(\bshe\b\W+\w+)", contents, re.I)
pronoun_next = re.findall(r"(\bs?he\b\W+\w+)", contents, re.I)
print(he_next[0:5], she_next[0:5], pronoun_next[0:5])

['he is', 'he had', 'he came', 'he agreed', 'he is'] ['she; "for', 'she told', 'she ought', 'she is', 'She was'] ['he is', 'he had', 'she; "for', 'she told', 'he came']


## Use capture groups to save the pronoun (he/she) as one match and the word as another

The first five should look like

```
[('he', 'is'),
 ('he', 'had'),
 ('she', 'told'),
 ('he', 'came'),
 ('he', 'agreed')]```

In [44]:
he_next_tuple = re.findall(r"\b(he)\b\W+(\w+)", contents, re.I)
she_next_tuple = re.findall(r"\b(she)\b\W+(\w+)", contents, re.I)
pronoun_next_tuple = re.findall(r"\b(s?he)\b\W+(\w+)", contents, re.I)
print(he_next_tuple[0:5], she_next_tuple[0:5], pronoun_next_tuple[0:5])

[('he', 'is'), ('he', 'had'), ('he', 'came'), ('he', 'agreed'), ('he', 'is')] [('she', 'for'), ('she', 'told'), ('she', 'ought'), ('she', 'is'), ('She', 'was')] [('he', 'is'), ('he', 'had'), ('she', 'for'), ('she', 'told'), ('he', 'came')]


## Save those matches into a dataframe

You can give the column names with `columns=['pronoun', 'verb']`

In [49]:
df = pd.DataFrame(pronoun_next_tuple, columns=['pronoun', 'verb'])
df.head()

,pronoun,verb
0,he,is
1,he,had
2,she,for
3,she,told
4,he,came


## How many times is each pronoun used?

In [50]:
df.pronoun.value_counts()

she    1384
he     1103
She     325
He      235
Name: pronoun, dtype: int64

## Oh, wait, clean that up.

Make it only 'he' and 'she' lowercase.

It should be about 1600 'she' and 1300 'he'

In [52]:
df['pronoun'] = df.pronoun.str.lower()
df.pronoun.value_counts()

she    1709
he     1338
Name: pronoun, dtype: int64

## What are the top 20 most common verbs?

In [62]:
df.groupby(by='verb').count().sort_values('pronoun', ascending = False).head(20)

,pronoun
verb,
had,372
was,372
could,172
is,140
would,94
has,72
did,69
will,50
might,46


## What are the top 20 most common verbs for 'he', and the top 20 most common for 'she'

**Tip:** Don't use groupby, just filter. If you want to know how, though, you can also look at "value counts for different categories" on [this page](http://jonathansoma.com/lede/foundations-2017/classes/more-pandas/class-notes/)

In [72]:
df[df['pronoun'] == 'he'].groupby(by='verb').count().sort_values('pronoun', ascending = False).head(20)

,pronoun,verb
0,he,is
2082,he,perceived
2076,he,had
2074,he,had
2067,he,was
2065,he,was
2063,he,first
2061,he,appear
2060,he,had
2059,he,came


In [70]:
df[df['pronoun'] == 'she'].groupby(by='verb').count().sort_values('pronoun', ascending = False).head(20)

,pronoun
verb,
was,212
had,205
could,132
is,66
would,59
did,39
felt,33
saw,29
will,26


## Who cries more, men or women? Give me a percentage answer.

**Tip:** It's `cried`, because of, you know, how books are written

In [65]:
df[df['verb'] == 'cried'].pronoun.value_counts()

she    11
he      1
Name: pronoun, dtype: int64

## How much more common is 'he' than 'she' in J.R.R. Tolkein's Fellowship of the Ring? How does that compare to Pride and Prejudice?

The book is in the same directory.

In [68]:
contents_jrr = open('data/Lord of the Rings - 01 - The Fellowship of the Ring - J. R. R. Tolkien - 1955.txt').read()
he_count_jrr = len(re.findall(r"(\bhe\b)", contents_jrr, re.I))
she_count_jrr = len(re.findall(r"(\bshe\b)", contents_jrr, re.I))
print('There are {she_count} "she"s and {he_count} "he"s in Austen\'s "Pride and Prejudice and {she_count_jrr} "she"s and {he_count_jrr} "he"s in Tolkien\'s Fellowship.'.format(he_count = he_count, she_count = she_count, he_count_jrr = he_count_jrr, she_count_jrr = she_count_jrr))

There are 1709 "she"s and 1338 "he"s in Austen's "Pride and Prejudice and 159 "she"s and 3062 "he"s in Tolkien's Fellowship.
